In [1]:
import pandas as pd
from pyvi import ViUtils

In [2]:
vt_df = pd.read_csv('../data/viettel.csv')
for column in vt_df.columns:
    print(column)

Tên sản phẩm
Gía
Màn hình
Camera sau
Camera trước
Chip
RAM
Bộ nhớ trong
Pin
SIM
Kích thước màn hình
Công nghệ màn hình


In [3]:
drop_columns = ['Màn hình', 'Camera sau', 'Camera trước']

for column in drop_columns:
    del vt_df[column]

vt_df.head()

,Tên sản phẩm,Gía,Chip,RAM,Bộ nhớ trong,Pin,SIM,Kích thước màn hình,Công nghệ màn hình
0,Samsung Galaxy S22 Ultra 12/256GB,31.990.000 ₫,Chip Snapdragon® 8 Gen 1 (4nm),12GB,256GB,Pin 5000 mAh | Sạc siêu nhanh 45W (Củ sạc 45W ...,2 nano sim hoặc 1 nano sim và 1 esim,6.8”,Dynamic AMOLED 2X
1,Samsung Galaxy S22 8/128GB,19.990.000 ₫,Chip Snapdragon® 8 Gen 1 (4nm),8GB,128GB,Pin 3700 mAh | Sạc siêu nhanh 25W (Củ sạc 25W ...,2 nano sim hoặc 1 nano sim và 1 esim,6.1”,Dynamic AMOLED 2X
2,Samsung Galaxy S22 Plus 8/128GB,23.990.000 ₫,Chip Snapdragon® 8 Gen 1 (4nm),8GB,128GB,Pin 4500 mAh | Sạc siêu nhanh 45W (Củ sạc 45W ...,2 nano sim hoặc 1 nano sim và 1 esim,6.6”,Dynamic AMOLED 2X
3,iPhone 13 Pro Max 128GB,29.490.000 ₫,Apple A15 Bionic 6 nhân; Tốc độ CPU 3.22 GHz; ...,6 GB,128 GB,4352 mAh; Li-Ion; Hỗ trợ sạc tối đa 20 W; Sạc ...,1 Nano SIM & 1 eSIM,"6.7""",OLED
4,iPhone 13 Pro Max 256GB,31.990.000 ₫,Apple A15 Bionic 6 nhân; Tốc độ CPU 3.22 GHz; ...,6 GB,256 GB,4352 mAh; Li-Ion; Hỗ trợ sạc tối đa 20 W; Sạc ...,1 Nano SIM & 1 eSIM,"6.7""",OLED


# Xử lý tên cột

In [4]:
vt_df = vt_df.rename(columns={"Tên sản phẩm": "ten", "Gía": "gia", 'Chip': 'cpu'})

for column in vt_df.columns:
    vt_df = vt_df.rename(columns={column: ViUtils.remove_accents(column).decode("utf-8").lower().strip().replace(' ', '_')})

# Xử lý trường tên

In [5]:
vt_df = vt_df[vt_df['ten'].notna()]
vt_df['ten'] = vt_df['ten'].apply(lambda s: ViUtils.remove_accents(s).decode("utf-8"))

# Xử lý trường giá

In [6]:
vt_df = vt_df[vt_df['gia'].notna()]
vt_df['gia'] = vt_df['gia'].apply(lambda s: str(s).replace('₫', '').replace('.', '').replace('*', '').strip())
vt_df['gia'] = vt_df['gia'].apply(lambda s: int(s))

# Xử lý trường kích thước màn hình

In [7]:
def process_screen_size(s):
    try:
        if " " in s:
            size = s.split(' ')
            size1 = float(size[0][:-1])
            size2 = float(size[1][:-1])
            size = size1 + size2
        else:
            size = float(s[:-1])
    except:
        size = 0
    return size

vt_df['kich_thuoc_man_hinh'] = vt_df['kich_thuoc_man_hinh'].apply(process_screen_size)

# Xử lý trường bộ nhớ trong

In [8]:
def process_rom(s):
    try:
        if 'GB' in s:
            return float(s.replace('GB', '').strip())
        elif 'MB' in s:
            return float(s.replace('MB', '').strip())/1024
        elif 'TB' in s:
            return float(s.replace('TB', '').strip())*1024
    except:
        return 0

vt_df['bo_nho_trong'] = vt_df['bo_nho_trong'].apply(process_rom)

# Xử lý trường RAM

In [9]:
def process_ram(s):
    try:
        return int(s.replace('GB', '').strip())
    except:
        return 0

vt_df['ram'] = vt_df['ram'].apply(process_ram)

In [10]:
for column in vt_df.columns:
    print(column)

ten
gia
cpu
ram
bo_nho_trong
pin
sim
kich_thuoc_man_hinh
cong_nghe_man_hinh


In [11]:
vt_df.to_csv('../clean_data/viettel.csv', index=False)